You are asked to write an expression processor for simple numeric expressions with the following constraints:

    Expressions use integral values (e.g., '13' ), single-letter variables defined in Variables, as well as + and - operators only

    There is no need to support braces or any other operations

    If a variable is not found in variables  (or if we encounter a variable with >1 letter, e.g. ab), the evaluator returns 0 (zero)

    In case of any parsing failure, evaluator returns 0

Example:

    calculate("1+2+3")  should return 6

    calculate("1+2+xy")  should return 0

    calculate("10-2-x")  when x=3 is in variables  should return 5

In [85]:
class Token:
    def __init__(self, value) -> None:
        self.value = value

    def __str__(self) -> str:
        return str(self.value)


class PlusToken(Token):
    def __init__(self) -> None:
        super().__init__(value="+")


class MinusToken(Token):
    def __init__(self) -> None:
        super().__init__(value="-")


class VariableToken(Token):
    def is_valid(self):
        return len(self.value) == 1


class DigitToken(Token):
    def __init__(self, value) -> None:
        super().__init__(int(value))


class BinaryExpression:
    def __init__(self) -> None:
        self.left = None
        self.right = None


class SumOperation(BinaryExpression):
    def __init__(self, left, right) -> None:
        self.left = left
        self.right = right

    @property
    def value(self):
        return self.left.value + self.right.value


class DiffOperation(BinaryExpression):
    def __init__(self, left, right) -> None:
        self.left = left
        self.right = right

    @property
    def value(self):
        return self.left.value - self.right.value


class ExpressionProcessor:
    def __init__(self):
        self.variables = {}
        self.error_message = None

    def calculate(self, expression):
        self.error_message = None
        self.tokens = self.lexing(expression)
        try:
            parsed_operation = self.parse(self.tokens)
        except ValueError as e:
            self.error_message = str(e)

            return 0

        return parsed_operation.value

    def parse(self, tokens):
        if len(tokens) == 1:
            token = tokens[0]
            if isinstance(token, VariableToken):
                if not token.is_valid:
                    raise ValueError("Found invalid variable named {}".format(token.value))

                if token.value not in self.variables:
                    raise ValueError("Variable {} not initialized".format(token.value))

                return DigitToken(self.variables[token.value])
            elif isinstance(token, DigitToken):
                return token
            else:
                raise ValueError("Malformed equation")

        for i, token in reversed(list(enumerate(tokens))):
            if isinstance(token, PlusToken):
                return SumOperation(self.parse(tokens[:i]), self.parse(tokens[i + 1 :]))
            elif isinstance(token, MinusToken):
                return DiffOperation(self.parse(tokens[:i]), self.parse(tokens[i + 1 :]))

    def lexing(self, expression):
        i = 0
        tokens = []

        while i < len(expression):
            if expression[i] == "+":
                tokens.append(PlusToken())
            elif expression[i] == "-":
                tokens.append(MinusToken())
            elif expression[i].isdigit():
                j = i + 1
                final_digit = expression[i]
                while j < len(expression):
                    if expression[j].isdigit():
                        final_digit += expression[j]
                        j += 1
                    else:
                        break

                tokens.append(DigitToken(final_digit))
                i = j - 1
            else:
                j = i + 1
                final_variable = expression[i]
                while j < len(expression):
                    if not (expression[j].isdigit() or expression[j] == "+" or expression[j] == "-"):
                        final_variable += expression[j]
                        j += 1
                    else:
                        break

                tokens.append(VariableToken(final_variable))
                i = j - 1

            i += 1

        return tokens

    def __str__(self) -> str:
        if self.tokens is None:
            return "No valid operation was computed"

        return " ".join(map(str, self.tokens))

In [86]:
exp = ExpressionProcessor()
exp.variables["x"] = 3

test1 = exp.calculate("1+2+3")
print("{} = {}".format(exp, test1))

test2 = exp.calculate("1+2+xy")
print("{} = {}".format(exp, test2))

test3 = exp.calculate("10-2-x")
print("{} = {}".format(exp, test3))

1 + 2 + 3 = 6
1 + 2 + xy = 0
10 - 2 - x = 5
